In [1]:
import pandas as pd
from pycaret.classification import *
from  sklearn.metrics import classification_report

In [81]:
df1 = pd.read_json('./newmodel/real/dataSet_Culture_10122023_2020_kirbi.json')
df2 = pd.read_json('./newmodel/real/dataSet_Culture_10122023_2021_kirbi.json')
df3 = pd.read_json('./newmodel/real/dataSet_Culture_10122023_2022_kirbi.json')
df4 = pd.read_json('./newmodel/real/dataSet_Culture_10122023_2023_kirbi.json')
df5 = pd.read_json('./newmodel/dataSet_Culture_21112023.json')

In [87]:
combined_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [88]:
combined_df['culture_name'].unique()

array(['Ячмень', 'Wheat', 'Морковь', 'Лук-Севок', 'Рапс', 'Люцерна',
       'Лён', 'Соя', 'Эспарцет', 'Potato', 'Вика', 'Донник',
       'Неизвестная Культура', 'Sunflower', 'Козлятник', 'Просо',
       'Тритикале', 'Сорго', 'Сидераты', 'Пшеница', 'Кукуруза', 'Свекла',
       'Томат', 'Картофель', 'Софлор', 'Лук', 'Клевер', 'Сил'],
      dtype=object)

In [89]:
crops = ['Ячмень', 'Wheat', 'Морковь', 'Лук-Севок', 'Рапс', 'Люцерна',

         'Лён', 'Соя', 'Эспарцет', 'Potato', 'Вика', 'Донник',

         'Неизвестная Культура', 'Sunflower', 'Козлятник', 'Просо',

         'Тритикале', 'Сорго', 'Сидераты', 'Пшеница', 'Кукуруза', 'Свекла',

         'Томат', 'Картофель', 'Софлор', 'Лук', 'Клевер', 'Сил']



# Translation dictionary for English names to Russian

translations = {

    'Wheat': 'Пшеница',

    'Potato': 'Картофель',

    'Sunflower': 'Подсолнечник'

}



# Translate the crop names to Russian

crops_russian = [translations.get(crop, crop) for crop in crops]



# Create a pandas Series from the list

crops_series = pd.Series(crops_russian)

crops_series

0                   Ячмень
1                  Пшеница
2                  Морковь
3                Лук-Севок
4                     Рапс
5                  Люцерна
6                      Лён
7                      Соя
8                 Эспарцет
9                Картофель
10                    Вика
11                  Донник
12    Неизвестная Культура
13            Подсолнечник
14               Козлятник
15                   Просо
16               Тритикале
17                   Сорго
18                Сидераты
19                 Пшеница
20                Кукуруза
21                  Свекла
22                   Томат
23               Картофель
24                  Софлор
25                     Лук
26                  Клевер
27                     Сил
dtype: object

In [90]:
combined_df['culture_name'] = combined_df['culture_name'].map(translations).fillna(combined_df['culture_name'])

combined_df['culture_name'].value_counts()

Пшеница                 122
Ячмень                   69
Кукуруза                 60
Свекла                   56
Люцерна                  44
Картофель                39
Эспарцет                 16
Соя                      10
Донник                    7
Неизвестная Культура      6
Сидераты                  3
Тритикале                 3
Подсолнечник              3
Вика                      3
Морковь                   3
Козлятник                 2
Лён                       2
Рапс                      2
Томат                     2
Лук                       2
Просо                     1
Сорго                     1
Лук-Севок                 1
Софлор                    1
Клевер                    1
Сил                       1
Name: culture_name, dtype: int64

In [94]:
combined_df.to_csv('kirbi_norm.csv', index=False)

In [107]:
# Counting the number of occurrences for each culture
culture_counts = combined_df['culture_name'].value_counts()

# Cultures with more than 5 members
cultures_more_than_5 = culture_counts[culture_counts > 5].index

# Filtering the dataset
filtered_data = combined_df[combined_df['culture_name'].isin(cultures_more_than_5)]

# Displaying the first few rows of the filtered dataset
filtered_data.head(), filtered_data['culture_name'].value_counts()


(   contour_id region  index_month_4  index_month_5  index_month_6  \
 0     16952.0   chui          0.383          0.492          0.261   
 1     16953.0   chui          0.522          0.684          0.537   
 2     16954.0   chui          0.592          0.610          0.429   
 6     16958.0   chui          0.050          0.225          0.147   
 8     16960.0   chui          0.112          0.190          0.289   
 
    index_month_7  index_month_8  elevation culture_name  
 0          0.065          0.071      634.0       Ячмень  
 1          0.162          0.091      633.0      Пшеница  
 2          0.140          0.096      625.0      Пшеница  
 6          0.087          0.057      637.0      Люцерна  
 8          0.514          0.546      621.0          Соя  ,
 Пшеница                 122
 Ячмень                   69
 Кукуруза                 60
 Свекла                   56
 Люцерна                  44
 Картофель                39
 Эспарцет                 16
 Соя                

In [108]:
unique_cultures = filtered_data['culture_name'].unique()

# Create a mapping from culture names to numeric values
culture_mapping = {culture: i+1 for i, culture in enumerate(unique_cultures)}

In [150]:
culture_mapping

{'Ячмень': 1,
 'Пшеница': 2,
 'Люцерна': 3,
 'Соя': 4,
 'Эспарцет': 5,
 'Картофель': 6,
 'Донник': 7,
 'Неизвестная Культура': 8,
 'Кукуруза': 9,
 'Свекла': 10}

In [109]:
filtered_data['culture_name'] = filtered_data['culture_name'].map(culture_mapping)


In [114]:
filtered_data_no_ik = filtered_data[filtered_data['region'] != 'ik']
df = filtered_data_no_ik.drop(columns=['contour_id', 'region'])

In [116]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)

# Perform KNN imputation
imputed_df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [120]:
df.isna().sum()

index_month_4     2
index_month_5    35
index_month_6     3
index_month_7    75
index_month_8     3
elevation        11
culture_name      0
dtype: int64

In [122]:
imputed_df['elevation'] = imputed_df['elevation'].astype(int)
imputed_df['culture_name'] = imputed_df['culture_name'].astype(int)

In [48]:
df = combined_df[(combined_df['culture_name'] == 2) | (combined_df['culture_name'] == 0)].reset_index(drop=True)
df = df.drop(columns=['contour_id'])
y_pshenica =  df.pop('culture_name')
y_pshenica_pred = model.predict(df)
df['y_pshenica'] = y_pshenica
df['y_pshenica_pred'] = y_pshenica_pred

In [68]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def calculate_evaluation_scores(y_true, y_pred, problem_type):
    scores = {}
    
    if problem_type == 'regression':
        scores['MAE'] = mean_absolute_error(y_true, y_pred)
        scores['MSE'] = mean_squared_error(y_true, y_pred)
        scores['R-squared'] = r2_score(y_true, y_pred)
    elif problem_type == 'classification':
        scores['Accuracy'] = accuracy_score(y_true, y_pred)
        scores['Precision'] = precision_score(y_true, y_pred,average='weighted')
        scores['Recall'] = recall_score(y_true, y_pred,average='weighted')
        scores['F1-Score'] = f1_score(y_true, y_pred,average='weighted')
        # You can add more classification metrics as needed
    
    return scores


scores_classification = calculate_evaluation_scores(df['y_pshenica'], df['y_pshenica_pred'], problem_type='classification')
print("\nClassification Scores:")
print(scores_classification)


Classification Scores:
{'Accuracy': 0.5396825396825397, 'Precision': 0.7453576025004597, 'Recall': 0.5396825396825397, 'F1-Score': 0.6251049078635285}


In [129]:
# Counting the number of occurrences for each culture
culture_counts = imputed_df['culture_name'].value_counts()

# Cultures with more than 5 members
cultures_more_than_5 = culture_counts[culture_counts > 5].index

# Filtering the dataset
filtered_data = imputed_df[imputed_df['culture_name'].isin(cultures_more_than_5)]

# Displaying the first few rows of the filtered dataset
filtered_data.head(), filtered_data['culture_name'].value_counts()

(   index_month_4  index_month_5  index_month_6  index_month_7  index_month_8  \
 0          0.383          0.492          0.261          0.065          0.071   
 1          0.522          0.684          0.537          0.162          0.091   
 2          0.592          0.610          0.429          0.140          0.096   
 3          0.050          0.225          0.147          0.087          0.057   
 4          0.112          0.190          0.289          0.514          0.546   
 
    elevation  culture_name  
 0        634             1  
 1        633             2  
 2        625             2  
 3        637             3  
 4        621             4  ,
 2     104
 1      60
 9      60
 10     56
 3      41
 6      33
 4      10
 Name: culture_name, dtype: int64)

In [131]:
filtered_data['culture_name'].value_counts()

2     104
1      60
9      60
10     56
3      41
6      33
4      10
Name: culture_name, dtype: int64

In [149]:
filtered_data['culture_name'].value_counts()

2     104
1      60
9      60
10     56
3      41
6      33
4      10
Name: culture_name, dtype: int64

In [139]:
from pycaret.classification import setup, compare_models

# Setting up PyCaret for classification
clf_setup = setup(data=filtered_data, 
                  target='culture_name', train_size=0.9)

,Description,Value
0,Session id,7916
1,Target,culture_name
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 4: 3, 6: 4, 9: 5, 10: 6"
4,Original data shape,"(364, 7)"
5,Transformed data shape,"(364, 7)"
6,Transformed train set shape,"(327, 7)"
7,Transformed test set shape,"(37, 7)"
8,Numeric features,6
9,Preprocess,True


In [140]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6364,0.9021,0.6364,0.6364,0.6286,0.5551,0.5595
1,0.5455,0.8921,0.5455,0.5406,0.5251,0.4304,0.4356
2,0.5455,0.8715,0.5455,0.4904,0.5093,0.4407,0.4489
3,0.5455,0.8712,0.5455,0.5274,0.5250,0.4356,0.4398
4,0.4242,0.8377,0.4242,0.4672,0.3940,0.2785,0.2873
5,0.5152,0.8426,0.5152,0.5247,0.5022,0.4034,0.4081
6,0.5455,0.8365,0.5455,0.6237,0.5430,0.4356,0.4446
7,0.4375,0.0000,0.4375,0.4375,0.4186,0.3035,0.3104
8,0.5000,0.8189,0.5000,0.5363,0.4994,0.3809,0.3833


In [141]:
rf_t = tune_model(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6667,0.9130,0.6667,0.6941,0.6675,0.5962,0.6016
1,0.5758,0.8785,0.5758,0.6406,0.5596,0.4809,0.4875
2,0.5152,0.8470,0.5152,0.5202,0.5019,0.4014,0.4051
3,0.5152,0.8727,0.5152,0.5639,0.5244,0.4146,0.4193
4,0.4242,0.8124,0.4242,0.4313,0.4033,0.2859,0.2932
5,0.4545,0.8045,0.4545,0.4578,0.4488,0.3363,0.3378
6,0.4545,0.8158,0.4545,0.6086,0.4415,0.3265,0.3406
7,0.3750,0.0000,0.3750,0.3684,0.3633,0.2390,0.2445
8,0.4375,0.7772,0.4375,0.4922,0.4538,0.3110,0.3129


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [142]:
f_rf = finalize_model(rf_t)

In [147]:
predict_model(f_rf)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,index_month_4,index_month_5,index_month_6,index_month_7,index_month_8,elevation,culture_name,prediction_label,prediction_score
282,0.066,-0.1070,0.137,0.0760,-0.073,625,1,1,0.59
315,0.046,0.4010,0.372,0.1455,0.146,1077,2,2,0.81
83,-0.049,0.2100,0.044,-0.0570,0.084,716,1,1,0.77
122,-0.115,0.3960,0.295,-0.2170,0.184,711,3,3,0.64
70,0.010,0.0470,0.061,0.0510,0.020,621,6,6,0.86
109,-0.058,0.0470,-0.043,-0.0200,0.235,587,10,10,0.70
319,0.007,-0.0040,0.009,0.4700,0.052,985,9,9,0.81
231,-0.027,-0.0010,-0.128,-0.1690,0.130,626,10,10,0.77
19,0.056,0.2460,0.286,0.1560,0.097,849,2,2,0.75
240,-0.033,0.0400,0.226,0.2000,-0.034,1253,2,2,0.82


In [145]:
save_model(f_rf, 'my_best_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['index_month_4', 'index_month_5',
                                              'index_month_6', 'index_month_7',
                                              'index_month_8', 'elevation'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               c...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None, max_features='sqrt',
                                         max_leaf_nodes=None, m